In [1]:
import numpy as np
import random as rm

# THEMES
Add different themes for the system to use, which will control the ranges and values that the musical and timbral parameters can take  
eg. lush, drone, busy, minimal, etc.  



- minimal- One note sustained  
- drone- long sustained chords, sawtooth wave heavy
- lush- full sounding, minimal effects and minimal chord selection
- busy- lots of chords, lots of effects, varying sustain for each note
- voice/choir synth- few chords, not as long sustained as drone/minimal

## Testing

In [2]:
# The statespace
states = ["Sleep","Icecream","Run"]

# Possible sequences of events
transitionName = [["SS","SR","SI"],["RS","RR","RI"],["IS","IR","II"]]

# Probabilities matrix (transition matrix)
transitionMatrix = [[0.2,0.6,0.2],[0.1,0.6,0.3],[0.2,0.7,0.1]]

In [3]:
if sum(transitionMatrix[0])+sum(transitionMatrix[1])+sum(transitionMatrix[1]) != 3:
    print("Somewhere, something went wrong. Transition matrix, perhaps?")
else: print("All is gonna be okay, you should move on!! ;)")


All is gonna be okay, you should move on!! ;)


In [4]:
# A function that implements the Markov model to forecast the state/mood.
def activity_forecast(days):
    # Choose the starting state
    activityToday = "Sleep"
    print("Start state: " + activityToday)
    # Shall store the sequence of states taken. So, this only has the starting state for now.
    activityList = [activityToday]
    i = 0
    # To calculate the probability of the activityList
    prob = 1
    while i != days:
        if activityToday == "Sleep":
            change = np.random.choice(transitionName[0],replace=True,p=transitionMatrix[0])
            if change == "SS":
                prob = prob * 0.2
                activityList.append("Sleep")
                pass
            elif change == "SR":
                prob = prob * 0.6
                activityToday = "Run"
                activityList.append("Run")
            else:
                prob = prob * 0.2
                activityToday = "Icecream"
                activityList.append("Icecream")
        elif activityToday == "Run":
            change = np.random.choice(transitionName[1],replace=True,p=transitionMatrix[1])
            if change == "RR":
                prob = prob * 0.5
                activityList.append("Run")
                pass
            elif change == "RS":
                prob = prob * 0.2
                activityToday = "Sleep"
                activityList.append("Sleep")
            else:
                prob = prob * 0.3
                activityToday = "Icecream"
                activityList.append("Icecream")
        elif activityToday == "Icecream":
            change = np.random.choice(transitionName[2],replace=True,p=transitionMatrix[2])
            if change == "II":
                prob = prob * 0.1
                activityList.append("Icecream")
                pass
            elif change == "IS":
                prob = prob * 0.2
                activityToday = "Sleep"
                activityList.append("Sleep")
            else:
                prob = prob * 0.7
                activityToday = "Run"
                activityList.append("Run")
        i += 1  
    print("Possible states: " + str(activityList))
    print("End state after "+ str(days) + " days: " + activityToday)
    print("Probability of the possible sequence of states: " + str(prob))

# Function that forecasts the possible state for the next 2 days
activity_forecast(2)


Start state: Sleep
Possible states: ['Sleep', 'Run', 'Run']
End state after 2 days: Run
Probability of the possible sequence of states: 0.3


# Note Selection

In [9]:
import numpy as np
import pandas as pd
from collections import Counter
np.random.seed(42)

# read file
data = pd.read_csv('../Liverpool_band_chord_sequence.csv')
# ----------------------------------------------------------------------
n = 3
chords = data['chords'].values
ngrams = zip(*[chords[i:] for i in range(n)])
bigrams = [" ".join(ngram) for ngram in ngrams]

# bigrams[:5]
# ----------------------------------------------------------------------
def predict_next_state(chord:str, data:list=bigrams, segment:int=1):
    """Predict next chord based on current state."""
    # create list of bigrams which stats with current chord
    bigrams_with_current_chord = [bigram for bigram in bigrams if bigram.split(' ')[0]==chord]
    # count appearance of each bigram
    count_appearance = dict(Counter(bigrams_with_current_chord))
    # convert apperance into probabilities
    for ngram in count_appearance.keys():
        count_appearance[ngram] = count_appearance[ngram]/len(bigrams_with_current_chord)
    
    # create list of possible options for the next chord
    options = [key.split(' ')[1] for key in count_appearance.keys()]
    # print(options)
    # create  list of probability distribution
    probabilities = list(count_appearance.values())
    # return random prediction
    return np.random.choice(options, p=probabilities)

# ----------------------------------------------------------------------
def generate_sequence(chord:str=None, data:list=bigrams, length:int=30):
    """Generate sequence of defined length."""
    # create list to store future chords
    chords = []
    chords.append(chord)
    for n in range(length):
        # append next chord for the list
        chords.append(predict_next_state(chord, bigrams))
        # use last chord in sequence to predict next chord
        chord = chords[-1]
    return chords


In [10]:
extended_chord_sequence = generate_sequence('C')

# Note duration

- Changes in note durations
- minumum note duration is 0.5s
- maximum note duration is 8 seconds
- To be modeled using a markov chain

In [11]:
# note_durations = [0.5, 0.75, 1.0, 1.5, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]
# The statespace
states = ["VS","S","M","L","VL"]
# Possible sequences of events
transitionName = [["VSVS","VSS","VSM","VSL","VSVL"],["SVS","SS","SM","SL","SVL"],["MVS","MS","MM","ML","MVL"],["LVS","LS","LM","LL","LVL"],["VLVS","VLS","VLM","VLL","VLVL"]]

# Probabilities matrix (transition matrix)
#TODO: Load transition matrix, last_note_min_length from each theme
transitionMatrix = np.array([
                             [0.2,0.2,0.2,0.2,0.2],
                             [0.2,0.2,0.2,0.2,0.2],
                             [0.2,0.2,0.2,0.2,0.2],
                             [0.2,0.2,0.2,0.2,0.2],
                             [0.2,0.2,0.2,0.2,0.2]
                             ])
assert sum(transitionMatrix[0])==1
assert sum(transitionMatrix[1])==1
assert sum(transitionMatrix[2])==1
assert sum(transitionMatrix[3])==1
assert sum(transitionMatrix[4])==1


def get_length_sequence(initial_state, n_steps, seed="NA"):
    sequence = [initial_state]
    if seed != "NA":
        np.random.seed(seed)
    for i in range(n_steps-1):
        transition_probs = transitionMatrix[states.index(sequence[-1])]
        next_state = np.random.choice(states, p=transition_probs)
        sequence.append(next_state)
    return sequence

def get_durations(total_length, sequence, last_note_min_length=3):
    length_reqd = total_length
    durations = []
    for i in range(len(sequence)):
        if sum(durations) > total_length:
            # print(durations)
            durations = durations[:-1]
            durations.append(round(total_length-sum(durations),2))
            # print(durations)
            # print(sum(durations))
            while durations[-1] < last_note_min_length:
                durations[-2] += durations[-1]
                durations=durations[:-1]
                # print(durations)
                # print(sum(durations))
            # print("exited while")
            # print(durations)
            # print(sum(durations))
            assert durations[-1] >= last_note_min_length
            assert round(sum(durations),2) == total_length
            # l = len(durations)
            return durations
            
        if i == len(sequence)-1:
            if length_reqd < last_note_min_length:
                durations.append(length_reqd)
                break
        if sequence[i] == "VS":
            durations.append(round(rm.uniform(0.5,1),2))
        elif sequence[i] == "S":
            durations.append(round(rm.uniform(1,2),2))
        elif sequence[i] == "M":
            durations.append(round(rm.uniform(2,3),2))
        elif sequence[i] == "L":
            durations.append(round(rm.uniform(3,5),2))
        elif sequence[i] == "VL":
            durations.append(round(rm.uniform(5,8),2))
        length_reqd -= durations[i]
    l = len(durations)
    assert durations[-1] >= last_note_min_length
    assert round(sum(durations),2) == total_length
    return durations

In [12]:
lengths = get_length_sequence("VL",5)
chord_sequence = extended_chord_sequence[:len(lengths)]
lengths

['VL', 'L', 'VS', 'VS', 'VL']

In [13]:
durs = get_durations(20,get_length_sequence("VL",20),2)
print(durs)
print(sum(durs))

[5.98, 7.47, 6.55]
20.0


# Note Density
- Number of simultaneous notes  
- Ranges from 1 to 5  
- Modeled using a markov chain  


Load transition matrix for note density based on theme

In [14]:
def get_note_density(sequence, initial_state=3, seed="NA", transition_probs=[0.2,0.3,0.3,0.1,0.1]):
    assert sum(transition_probs) == 1
    states = [1,2,3,4,5]
    densities=[initial_state]
    for i in range(len(sequence)-1):
        densities.append(np.random.choice(states, p=transition_probs))
    return densities

In [17]:
get_note_density(extended_chord_sequence)

[3,
 4,
 3,
 5,
 1,
 1,
 1,
 2,
 2,
 2,
 4,
 2,
 2,
 3,
 1,
 4,
 1,
 5,
 3,
 1,
 1,
 4,
 3,
 3,
 3,
 1,
 2,
 1,
 4,
 3,
 2]

# Melodic Range
- Number of octaves to select from (Range: 1 - 3)
- Specific octave to use (Changes from one timbre to another)
- Modeled using random sampling with probabilities

In [18]:
def get_octaves(sequence, initial_state=3, seed="NA", transition_probs=[0.3,0.5,0.1,0.1]):
    assert sum(transition_probs) == 1
    octaves=[initial_state]
    for i in range(len(sequence)-1):
        octaves.append(np.random.choice([2,3,4,5], p=transition_probs))
    return octaves

In [19]:
get_octaves(chord_sequence)

[3, 2, 3, 3, 3]

# Loudness
- Scaling factor for the final output (Range: 0 - 1)
- Initial and final values to be selected and connected using a Gensound Linear or Exponential object

# Melodic Direction
- Ascending or descending based on the selected segment

# Combined params

In [20]:
def get_chord_and_metadata(total_length, ending_chord_length,starting_chord='C', starting_length="VL", starting_density=1, starting_octave=3, seed="NA"):
    chord_sequence = generate_sequence('C')
    lengths = get_length_sequence(starting_length, 5, seed)
    durations = get_durations(total_length,lengths,ending_chord_length)
    chord_sequence = chord_sequence[:len(durations)]

    densities = get_note_density(chord_sequence, starting_density, seed)
    octaves = get_octaves(chord_sequence)

    assert durations[-1] >= ending_chord_length
    return {'chords':chord_sequence, 'durations': durations, 'note_densities':densities, 'octaves':octaves}

In [21]:
get_chord_and_metadata(20,2)

AssertionError: 

# Working with musical_params.py

In [1]:
import musical_params

/Users/vedant/Desktop/Programming/emotional-background-music-generation/src


In [1]:
import numpy as np
import pandas as pd
import random as rm
from collections import OrderedDict

from collections import Counter
major_keys = OrderedDict({
                'F':['Bb','F','C',"Dm"], # Each list contains keys that it can transition to (the first 3 are major and the last is minor)
                'C':['F','C','G',"Am"],
                'G':['C','G','D',"Em"],
                'D':['G','D','A',"Bm"],
                'A':['D','A','E',"F#m"],
                'E':['A','E','B',"C#m"],
                'B':['E','B','Gb',"G#m"],
                'Gb':['B','Gb','Db',"Ebm"],
                'Db':['Gb','Db','Ab',"Bbm"], 
                'Ab':['Db','Ab','Eb',"Fm"], 
                'Eb':['Ab','Eb','Bb',"Cm"], 
                'Bb':['Eb','Bb','F',"Gm"],

                'Am':["C","C","C","Dm"], # Minor keys can transition back to the major, or an adjacent minor 10% of the time
                'Em':["G","G","G","Bm"],
                'Bm':["D","D","D","F#m"],
                'F#m':["A","A","A","C#m"],
                'C#m':["Eb","Eb","Eb","G#m"],
                'G#m':["B","B","B","Ebm"],
                'Ebm':["Gb","Gb","Gb","Bbb"], 
                'Bbm':["Db","Db","Db","Fm"], 
                'Fm':["Ab","Ab","Ab","Cm"], 
                'Cm':["Eb","Eb","Eb","Gm"],
                'Gm':["Bb","Bb","Bb","Dm"],
                'Dm':["F","F","F","Am"]
            })
minor_keys = OrderedDict({
                'Am':['Dm','Am','Em',"C"],
                'Em':['Am','Em','Bm',"G"],
                'Bm':['Em','Bm','F#m',"D"],
                'F#m':['Bm','F#m','C#m',"A"],
                'C#m':['F#m','C#m','G#m',"Eb"],
                'G#m':['C#m','G#m','Ebm',"B"],
                'Ebm':['G#m','Ebm','Bbm',"Gb"], 
                'Bbm':['Ebm','Bbm','Fm',"Db"], 
                'Fm':['Bbm','Fm','Cm',"Ab"], 
                'Cm':['Fm','Cm','Gm',"Eb"],
                'Gm':['Cm','Gm','Dm',"Bb"],
                'Dm':['Gm','Dm','Am',"F"],

                'F':["Dm","Dm","Dm","C"], # Major keys can transition back to the minor, or an adjacent Major 10% of the time
                'C':["Am","Am","Am","G"],
                'G':["Em","Em","Em","D"],
                'D':["Bm","Bm","Bm","A"],
                'A':["F#m","F#m","F#m","E"],
                'E':["C#m","C#m","C#m","B"],
                'B':["G#m","G#m","G#m","Gb"],
                'Gb':["Ebm","Ebm","Ebm","Db"],
                'Db':["Bbm","Bbm","Bbm","Ab"], 
                'Ab':["Fm","Fm","Fm","Eb"], 
                'Eb':["Cm","Cm","Cm","Bb"], 
                'Bb':["Gm","Gm","Gm","F"]
                })

transition_probabilities=[0.3,0.3,0.3,0.1]

def generate_sequence_v2(starting_key, length):
    """Generate sequence of defined length."""
    # create list to store future chords
    chords = []
    current_key = starting_key
    chords.append(current_key)
    for i in range(length-1):
        composition_type = rm.choice(['nearest','circle-of-fifths'])
    
        
    return chords

In [4]:
print(list(range(21,108+1)))

[21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108]


In [6]:
import random as rm
rm.choice([[0],[4],[7]])

[0]

In [8]:
[1,5,8,13,rm.choice([20,-5])]

[1, 5, 8, 13, -5]

In [106]:
def get_note_density(sequence, initial_state=3, seed="NA", transition_probs=[0.2,0.3,0.3,0.1,0.1]):
    assert sum(transition_probs) == 1
    states = [1,2,3,4,5]
    densities=[initial_state]
    for i in range(len(sequence)-1):
        densities.append(np.random.choice(states, p=transition_probs))
    return densities

midi_notes = list(range(21,108+1))

def get_major_scale(midi_note):
    return [midi_note, midi_note + 2, midi_note + 4, midi_note + 5, midi_note + 7, midi_note + 9, midi_note + 11, midi_note + 12]

def get_minor_scale(midi_note):
    return [midi_note, midi_note + 2, midi_note + 3, midi_note + 5, midi_note + 7, midi_note + 8, midi_note + 10, midi_note + 12]

def get_roots_search_space(midi_note):
    major_scale = get_major_scale(midi_note)
    # print (major_scale)
    minor_scale = get_minor_scale(midi_note)
    # print(minor_scale)
    major_roots = np.array([major_scale[3-1], major_scale[-3]-12])
    # print (major_roots)
    minor_roots = np.array([minor_scale[3-1], minor_scale[-3]-12])
    # print(minor_roots)
    return np.append(major_roots, minor_roots)

def get_major_intervals(num_notes):
    match num_notes:
        case 1:
            return np.array(rm.choice([[1],[5]])) # root or fifth
        case 2:
            return np.array([1,8]) # root,fifth
        case 3:
            return np.array([1,5,8]) # root, third, fifth
        case 4:
            return np.array([1,5,8,13]) # root, third, fifth, octave
        case 5:
            return np.array([1,5,8,13,rm.choice([20,-4])]) # root, third, fifth, octave, fifth of higher/lower octave

def get_minor_intervals(num_notes):
    match num_notes:
        case 1:
            return np.array(rm.choice([[1],[4]])) # root or fifth
        case 2:
            return np.array([1,8]) # root,fifth
        case 3:
            return np.array([1,4,8]) # root, third, fifth
        case 4:
            return np.array([1,4,8,13]) # root, third, fifth, octave
        case 5:
            return np.array([1,4,8,13,rm.choice([20,-4])]) # root, third, fifth, octave, fifth of higher/lower octave

def get_chord_search_space(midi_note, num_notes=3):
    roots = get_roots_search_space(midi_note)
    major_interval = get_major_intervals(num_notes)
    minor_interval = get_minor_intervals(num_notes)
    major_chords = []
    minor_chords = []
    for root in roots:
        major_chords.append(root + major_interval)
        minor_chords.append(root + minor_interval)
    # print(major_chords)
    # print(minor_chords)
    # print([number_to_note(num)[0] for num in major_chords])
    # print([number_to_note(num)[0] for num in minor_chords])
    return np.append(major_chords, minor_chords, axis=0)

NOTES = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
OCTAVES = list(range(11))
NOTES_IN_OCTAVE = len(NOTES)

def number_to_note(number: int) -> tuple:
    octave = number // NOTES_IN_OCTAVE
    # assert octave in OCTAVES
    # assert 0 <= number <= 127
    note = NOTES[number % NOTES_IN_OCTAVE]

    return number % NOTES_IN_OCTAVE, note, octave # Note number (0-11), note name, octave

def note_to_number(note: str, octave: int) -> int:
    assert note in NOTES
    # assert octave in OCTAVES

    note = NOTES.index(note)
    note += (NOTES_IN_OCTAVE * octave)

    # assert 0 <= note <= 127

    return note

def flatten(l):
    return [item for sublist in l for item in sublist]

def get_next_chord(current_chord):
    sorted_current_chord=sorted([number_to_note(note)[0] for note in current_chord])
    possible_chords=[]
    nearest_chords=[]
    chords_midi=get_chord_search_space(current_chord[0])
    for chord in get_chord_search_space(current_chord[0]):
        possible_chords.append([number_to_note(note)[0] for note in chord])
        for i in range(len(current_chord)):
            mask = np.count_nonzero(np.roll(np.array([number_to_note(note)[0] for note in chord]),i) - sorted_current_chord) == 1
            if np.all(mask) != False:
                # print(chord)
                nearest_chords.append(chord)
    return rm.choice(nearest_chords), np.array(nearest_chords)

In [107]:
print(f"Current Chord: {[number_to_note(note)[1] for note in current_chord]}")
for chord in get_next_chord(current_chord)[1]:
    print("Suggested chords: ",[number_to_note(note)[1] for note in chord])

Current Chord: ['C', 'E', 'G']
Suggested chords:  ['E', 'G', 'B']
Suggested chords:  ['A', 'C', 'E']


In [110]:
Cmaj = [60, 64, 67]
Amin = [57, 60, 64]
current_chord = Amin
note_names = ['C','C#','D','D#','E','F','F#','G','G#','A','A#','B']
for ele in get_chord_search_space(current_chord[0]):
    # print(ele)
    print([number_to_note(note)[1] for note in ele])

['D', 'F#', 'A']
['G', 'B', 'D']
['C#', 'F', 'G#']
['F#', 'A#', 'C#']
['D', 'F', 'A']
['G', 'A#', 'D']
['C#', 'E', 'G#']
['F#', 'A', 'C#']


In [360]:
current_chord[0]

60

In [3]:
d = {0.0:1, 30.5:2, 60:5}
list(d.keys())

[0.0, 30.5, 60]

In [4]:
l = list(d.keys())
for i in range(len(l)-1):
    print(f"{l[i]}-{l[i+1]}")

0.0-30.5
30.5-60


In [9]:
mood_map = d
total_time = 80
emotion_time_intervals = []
l = list(mood_map.keys())
for i in range(len(l)-1):
    emotion_time_intervals.append((l[i], l[i+1]))
if l[i+1] < total_time:
    emotion_time_intervals.append((l[i+1], total_time))
emotion_time_intervals

[(0.0, 30.5), (30.5, 60), (60, 80)]

In [10]:
for i,t in enumerate(emotion_time_intervals):
    print(f"Emotion {i+1}: {mood_map[t[0]]} from {t[0]} to {t[1]}")

Emotion 1: 1 from 0.0 to 30.5
Emotion 2: 2 from 30.5 to 60
Emotion 3: 5 from 60 to 80


In [7]:
dur=0
for interval in emotion_time_intervals:
    dur += interval[1] - interval[0]
dur == emotion_time_intervals[-1][1]

True

80.0

### Creating a hashmap for distances between notes on major scale

In [222]:
major_interval = get_major_intervals(3)
minor_interval = get_minor_intervals(3)
cols=[]
for root in range(21,108+1):
    # print(root, [root+i for i in interval])
    cols.append([root+i for i in major_interval])
    cols.append([root+1 for i in minor_interval])

In [308]:
def midi_to_note_name(midi):
    """Convert midi note to note name."""
    note_names = ['C','C#','D','D#','E','F','F#','G','G#','A','A#','B']
    # octave = (midi-21)//12
    note = (midi-21)%12
    return note#note_names[note]
def flatten(l):
    return [item for sublist in l for item in sublist]


In [321]:
def get_closest_chord(current_chord):
    current_chord=sorted([midi_to_note_name(note) for note in current_chord])
    major_interval = get_major_intervals(3)
    minor_interval = get_minor_intervals(3)
    l=[]
    note_rep=[]
    cols=[]
    selected_note_rep=[]
    for root in range(21,108+1):
        cols.append([root+i for i in major_interval])
        cols.append([root+i for i in minor_interval])
        note_rep.append(sorted([midi_to_note_name(root+i) for i in major_interval]))
        note_rep.append(sorted([midi_to_note_name(root+i) for i in minor_interval]))
    chord_cmp = np.array(note_rep) - current_chord
    for i in range(len(current_chord)+1):
        chord_cmp = np.roll(np.array(note_rep) - current_chord, i, axis=1)
        l.append(np.array(cols)[np.count_nonzero(chord_cmp, axis=1) == 1])

        selected_note_rep.append(np.array(note_rep)[np.count_nonzero(chord_cmp, axis=1) == 1])
    return np.array(flatten(l)), np.array(flatten(selected_note_rep))
    

In [329]:
Cmaj = [60, 64, 67]
Amin = [57, 60, 64]
cmajsorted = np.array(sorted([midi_to_note_name(note) for note in Cmaj]))
aminsorted = np.array(sorted([midi_to_note_name(note) for note in Amin]))